# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np 
import pandas as pd
from scipy import stats
import math
import requests

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [51]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks
from sec import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN = 'sk_445a529ea78f4d4d9762bc72be89c051'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [52]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2958254272160,
 'week52high': 194.48,
 'week52low': 123.64,
 'week52highSplitAdjustOnly': 194.48,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.3005890268678679,
 'sharesOutstanding': 15728702000,
 'float': 0,
 'avg10Volume': 46151625,
 'avg30Volume': 54803426,
 'day200MovingAvg': 161.02,
 'day50MovingAvg': 183.99,
 'employees': 164000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.9264396314120724,
 'dividendYield': 0.004925774305678819,
 'nextDividendDate': '',
 'exDividendDate': '2023-05-12',
 'nextEarningsDate': '2023-08-03',
 'peRatio': 31.36368647660648,
 'beta': 1.2669794945491828,
 'maxChangePercent': 72.22561806501851,
 'year5ChangePercent': 3.2010650083985563,
 'year2ChangePercent': 0.3147804446681417,
 'year1ChangePercent': 0.3094766479681461,
 'ytdChangePercent': 0.45377193836773766,
 'month6ChangePercent': 0.4150024262988224,
 'month3ChangePercent': 0.17288958761157214,
 'month1ChangePercent': 0.03934571175950485,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [53]:
data['year1ChangePercent']

0.3094766479681461

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [54]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]  

symbol_groups = list( chunks(stocks['Symbol'],100))
symbol_strings = []
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns= my_columns)
print(symbol_strings)



['MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AMD,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ANET,AJG,AIZ,T,ATO,ADSK,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BBWI,BAX,BDX,WRB,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BG,CHRW,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC', 'CNP,CDAY,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DIS,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FITB,FSLR', 'FE,FIS,FI,FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GL,GPN,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [55]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        final_dataframe.loc[len(final_dataframe)] = [symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A']
        
        
final_dataframe
    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,101.570,-0.169403,N/A
1,AOS,73.480,0.317146,N/A
2,ABT,106.855,0.005814,N/A
3,ABBV,134.960,-0.080936,N/A
4,ACN,313.670,0.144948,N/A
...,...,...,...,...
498,YUM,135.050,0.167649,N/A
499,ZBRA,315.850,0.059856,N/A
500,ZBH,142.650,0.396770,N/A
501,ZION,31.285,-0.360592,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [56]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe= final_dataframe[:50]

In [57]:
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,409,RCL,99.245,2.107702,N/A
1,349,NVDA,438.950,1.801243,N/A
2,199,FSLR,195.015,1.736614,N/A
3,334,NFLX,444.010,1.482294,N/A
4,218,GE,111.210,1.291523,N/A
5,91,CCL,18.230,1.226148,N/A
6,57,AXON,195.890,1.089084,N/A
7,495,WYNN,108.830,1.059314,N/A
8,346,NCLH,21.785,1.044708,N/A
9,194,FICO,811.010,0.934828,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [58]:
def portfolio_input():
    global portfolio_size
    portfolio_size = int(input('Enter the size of your portfolio: ')) 
    
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 50000
50000


In [59]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,409,RCL,99.245,2.107702,10
1,349,NVDA,438.950,1.801243,2
2,199,FSLR,195.015,1.736614,5
3,334,NFLX,444.010,1.482294,2
4,218,GE,111.210,1.291523,8
5,91,CCL,18.230,1.226148,54
6,57,AXON,195.890,1.089084,5
7,495,WYNN,108.830,1.059314,9
8,346,NCLH,21.785,1.044708,45
9,194,FICO,811.010,0.934828,1


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [67]:
hqm_columns = [
    'Ticker','Price', 'Number of Shares to Buy', 'One-Year Price Return', 'One-Year Return Percentile',
    'Six-month Price Return', 'Six-month Return Percentile',
    'Three-month Price Return', 'Three-month Return Percentile',
    'One-Month Price Return', 'One-Month Return Percentile',
]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        hqm_dataframe.loc[len(hqm_dataframe)] = [symbol,data[symbol]['price'],'NA',data[symbol]['stats']['year1ChangePercent'],'NA',data[symbol]['stats']['month6ChangePercent'],'NA',data[symbol]['stats']['month3ChangePercent'],'NA',data[symbol]['stats']['month1ChangePercent'],'NA']
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,101.570,NA,-0.169403,NA,-0.188048,NA,-0.010432,NA,0.020220,NA
1,AOS,73.480,NA,0.317146,NA,0.180794,NA,0.121519,NA,0.064108,NA
2,ABT,106.855,NA,0.005814,NA,-0.040863,NA,0.045960,NA,0.057134,NA
3,ABBV,134.960,NA,-0.080936,NA,-0.121221,NA,-0.152205,NA,-0.018888,NA
4,ACN,313.670,NA,0.144948,NA,0.106220,NA,0.085937,NA,0.003239,NA
...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.050,NA,0.167649,NA,0.039372,NA,0.024520,NA,0.005471,NA
499,ZBRA,315.850,NA,0.059856,NA,0.031255,NA,0.016873,NA,0.125610,NA
500,ZBH,142.650,NA,0.396770,NA,0.155822,NA,0.091628,NA,0.054467,NA
501,ZION,31.285,NA,-0.360592,NA,-0.381459,NA,0.046259,NA,-0.014415,NA


In [69]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,101.570,NA,-0.169403,9.741551,-0.188048,NA,-0.010432,NA,0.020220,NA
1,AOS,73.480,NA,0.317146,NA,0.180794,NA,0.121519,NA,0.064108,NA
2,ABT,106.855,NA,0.005814,NA,-0.040863,NA,0.045960,NA,0.057134,NA
3,ABBV,134.960,NA,-0.080936,NA,-0.121221,NA,-0.152205,NA,-0.018888,NA
4,ACN,313.670,NA,0.144948,NA,0.106220,NA,0.085937,NA,0.003239,NA
...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.050,NA,0.167649,NA,0.039372,NA,0.024520,NA,0.005471,NA
499,ZBRA,315.850,NA,0.059856,NA,0.031255,NA,0.016873,NA,0.125610,NA
500,ZBH,142.650,NA,0.396770,NA,0.155822,NA,0.091628,NA,0.054467,NA
501,ZION,31.285,NA,-0.360592,NA,-0.381459,NA,0.046259,NA,-0.014415,NA


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [71]:
time_periods = [
    'One-Year','Six-month','Three-month','One-Month'   
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row,percentile_col] = stats.percentileofscore(hqm_dataframe[change_col],hqm_dataframe.loc[row,change_col])
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,101.570,NA,-0.169403,9.741551,-0.188048,7.753479,-0.010432,32.604374,0.020220,30.815109
1,AOS,73.480,NA,0.317146,75.944334,0.180794,82.50497,0.121519,74.15507,0.064108,64.612326
2,ABT,106.855,NA,0.005814,30.815109,-0.040863,32.405567,0.045960,49.701789,0.057134,59.045726
3,ABBV,134.960,NA,-0.080936,17.892644,-0.121221,15.904573,-0.152205,4.373757,-0.018888,9.542744
4,ACN,313.670,NA,0.144948,53.081511,0.106220,69.582505,0.085937,61.431412,0.003239,20.27833
...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.050,NA,0.167649,56.262425,0.039372,54.671968,0.024520,42.743539,0.005471,21.868787
499,ZBRA,315.850,NA,0.059856,40.357853,0.031255,52.882704,0.016873,40.159046,0.125610,92.047714
500,ZBH,142.650,NA,0.396770,81.510934,0.155822,79.522863,0.091628,63.419483,0.054467,57.455268
501,ZION,31.285,NA,-0.360592,1.590457,-0.381459,0.596421,0.046259,50.099404,-0.014415,11.530815


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [75]:
hqm_dataframe['HQM_Score'] = 'N/A'

In [76]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,MMM,101.570,NA,-0.169403,9.741551,-0.188048,7.753479,-0.010432,32.604374,0.020220,30.815109,N/A
1,AOS,73.480,NA,0.317146,75.944334,0.180794,82.50497,0.121519,74.15507,0.064108,64.612326,N/A
2,ABT,106.855,NA,0.005814,30.815109,-0.040863,32.405567,0.045960,49.701789,0.057134,59.045726,N/A
3,ABBV,134.960,NA,-0.080936,17.892644,-0.121221,15.904573,-0.152205,4.373757,-0.018888,9.542744,N/A
4,ACN,313.670,NA,0.144948,53.081511,0.106220,69.582505,0.085937,61.431412,0.003239,20.27833,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.050,NA,0.167649,56.262425,0.039372,54.671968,0.024520,42.743539,0.005471,21.868787,N/A
499,ZBRA,315.850,NA,0.059856,40.357853,0.031255,52.882704,0.016873,40.159046,0.125610,92.047714,N/A
500,ZBH,142.650,NA,0.396770,81.510934,0.155822,79.522863,0.091628,63.419483,0.054467,57.455268,N/A
501,ZION,31.285,NA,-0.360592,1.590457,-0.381459,0.596421,0.046259,50.099404,-0.014415,11.530815,N/A


In [77]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM_Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,MMM,101.570,NA,-0.169403,9.741551,-0.188048,7.753479,-0.010432,32.604374,0.020220,30.815109,20.228628
1,AOS,73.480,NA,0.317146,75.944334,0.180794,82.50497,0.121519,74.15507,0.064108,64.612326,74.304175
2,ABT,106.855,NA,0.005814,30.815109,-0.040863,32.405567,0.045960,49.701789,0.057134,59.045726,42.992048
3,ABBV,134.960,NA,-0.080936,17.892644,-0.121221,15.904573,-0.152205,4.373757,-0.018888,9.542744,11.928429
4,ACN,313.670,NA,0.144948,53.081511,0.106220,69.582505,0.085937,61.431412,0.003239,20.27833,51.093439
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,135.050,NA,0.167649,56.262425,0.039372,54.671968,0.024520,42.743539,0.005471,21.868787,43.88668
499,ZBRA,315.850,NA,0.059856,40.357853,0.031255,52.882704,0.016873,40.159046,0.125610,92.047714,56.361829
500,ZBH,142.650,NA,0.396770,81.510934,0.155822,79.522863,0.091628,63.419483,0.054467,57.455268,70.477137
501,ZION,31.285,NA,-0.360592,1.590457,-0.381459,0.596421,0.046259,50.099404,-0.014415,11.530815,15.954274


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [86]:
hqm_dataframe.sort_values('HQM_Score',ascending=False,inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)


ValueError: cannot insert level_0, already exists

In [89]:
hqm_dataframe.drop(['index'],axis=1,inplace=True)
hqm_dataframe

/var/folders/wk/48ywwld12k3b18wmjf4yr1080000gn/T/ipykernel_38137/919137605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.drop(['index'],axis=1,inplace=True)


,level_0,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,0,CCL,18.230,0,1.226148,99.005964,0.916836,99.403579,0.882470,100.0,0.443850,100.0,99.602386
1,1,NCLH,21.785,0,1.044708,98.409543,0.537037,98.011928,0.655096,99.801193,0.305183,99.801193,99.005964
2,2,META,309.210,0,0.831348,97.415507,1.244638,99.801193,0.394856,98.409543,0.125835,92.246521,96.968191
3,3,RCL,99.245,0,2.107702,100.0,0.710811,99.005964,0.596469,99.602386,0.110283,87.673956,96.570577
4,4,DAL,48.010,0,0.661318,94.831014,0.274489,90.258449,0.407172,98.60835,0.238798,99.204771,95.725646
5,5,AVGO,889.540,0,0.890765,98.011928,0.553068,98.210736,0.434230,98.807157,0.107945,86.878728,95.477137
6,6,NVDA,438.950,0,1.801243,99.801193,1.651146,100.0,0.561109,99.403579,0.093758,81.11332,95.079523
7,7,PANW,232.670,0,0.455079,86.083499,0.830371,99.204771,0.297806,96.620278,0.136892,94.035785,93.986083
8,8,GWW,777.050,0,0.689665,95.228628,0.373918,94.831014,0.230604,92.842942,0.126288,92.445328,93.836978
9,9,URI,467.190,0,0.848219,97.813121,0.200137,84.294235,0.239547,93.240557,0.175569,98.011928,93.33996


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [80]:
portfolio_input()

Enter the size of your portfolio: 10000000


In [90]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

,level_0,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-month Price Return,Six-month Return Percentile,Three-month Price Return,Three-month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM_Score
0,0,CCL,18.230,10970,1.226148,99.005964,0.916836,99.403579,0.882470,100.0,0.443850,100.0,99.602386
1,1,NCLH,21.785,9180,1.044708,98.409543,0.537037,98.011928,0.655096,99.801193,0.305183,99.801193,99.005964
2,2,META,309.210,646,0.831348,97.415507,1.244638,99.801193,0.394856,98.409543,0.125835,92.246521,96.968191
3,3,RCL,99.245,2015,2.107702,100.0,0.710811,99.005964,0.596469,99.602386,0.110283,87.673956,96.570577
4,4,DAL,48.010,4165,0.661318,94.831014,0.274489,90.258449,0.407172,98.60835,0.238798,99.204771,95.725646
5,5,AVGO,889.540,224,0.890765,98.011928,0.553068,98.210736,0.434230,98.807157,0.107945,86.878728,95.477137
6,6,NVDA,438.950,455,1.801243,99.801193,1.651146,100.0,0.561109,99.403579,0.093758,81.11332,95.079523
7,7,PANW,232.670,859,0.455079,86.083499,0.830371,99.204771,0.297806,96.620278,0.136892,94.035785,93.986083
8,8,GWW,777.050,257,0.689665,95.228628,0.373918,94.831014,0.230604,92.842942,0.126288,92.445328,93.836978
9,9,URI,467.190,428,0.848219,97.813121,0.200137,84.294235,0.239547,93.240557,0.175569,98.011928,93.33996
